In [1]:
using Distributed

In [2]:
M = 100
angle_array = []
for i = 2π/M:2π/M:2π
    append!(angle_array,i)
end

In [3]:
addprocs(30);

In [6]:
using Statistics
using BenchmarkTools
using LinearAlgebra
using Distributed
using ProgressMeter
using FFTW
using MLDatasets
using Images
using Interpolations
using HDF5
using IterTools
push!(LOAD_PATH, pwd())
using DHC_2DUtils

In [7]:
L = 8
rand_mat = rand(16,16)
filter_hash = fink_filter_hash(1,L,nx=16,wd=2)
temp = DHC_compute(rand_mat,filter_hash);

In [8]:
train_x, train_y = MNIST.traindata()
test_x, test_y   = MNIST.testdata()

lst_train = Array{Any}(undef, 0)
for i = 1:60000
    push!(lst_train,train_x[:,:,i])
end


lst_test = Array{Any}(undef, 0)
for i = 1:10000
    push!(lst_test,test_x[:,:,i])
end

In [9]:
M = 100
train_angles = [2π/M,π/3,2π/3,3π/3,4π/3,5π/3];

In [10]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(256,256))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_ang_1_2.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 1:09:31


In [11]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,256,256)
        impad[142:-1:115,115:142] = im'
        imbig = imresize(impad,(256,256))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_ang_3_0.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 2:03:36


In [12]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,32,32)
        impad[30:-1:3,3:30] = im'
        imbig = imresize(impad,(256,256))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(angle_array,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_ang_0_3.h5", "main/data", mnist_DHC_out)

Progress:  40%|████████████████▋                        |  ETA: 3:35:18

RemoteException: InterruptException:
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:851
deserialize_fillarray!(::Array{Any,1}, ::Distributed.ClusterSerializer{Sockets.TCPSocket}) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:1094
deserialize_array(::Distributed.ClusterSerializer{Sockets.TCPSocket}) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:1086
handle_deserialize(::Distributed.ClusterSerializer{Sockets.TCPSocket}, ::Int32) at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:773
deserialize at /scratch/pkrastev/lmod_build/julia-1.3.1/usr/share/julia/stdlib/v1.3/Serialization/src/Serialization.jl:722 [inlined]
deserialize_msg(::Distributed.ClusterSerializer{Sockets.TCPSock

In [13]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=3)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("../scratch_AKS/mnist_DHC_train_ang_1_1_wd3.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:43:16
HDF5-DIAG: Error detected in HDF5 (1.12.0) thread 0:
  #000: H5F.c line 705 in H5Fcreate(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3393 in H5VL_file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #002: H5VLcallback.c line 3358 in H5VL__file_create(): file create failed
    major: Virtual Object Layer
    minor: Unable to create file
  #003: H5VLnative_file.c line 65 in H5VL__native_file_create(): unable to create file
    major: File accessibility
    minor: Unable to open file
  #004: H5Fint.c line 1568 in H5F_open(): unable to open file: name = '../scratch_AKS/mnist_DHC_train_ang_1_1_wd3.h5', tent_flags = 13
    major: File accessibility
    minor: Unable to open file
  #005: H5FD.c line 741 in H5FD_open(): open failed
    major: Virtual File Layer
    minor: Unable to initialize object
  #006:

ErrorException: Error creating file ../scratch_AKS/mnist_DHC_train_ang_1_1_wd3.h5

In [14]:
h5write("mnist_DHC_train_ang_1_1_wd3.h5", "main/data", mnist_DHC_out)

In [15]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=128,wd=1)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,64,64)
        impad[46:-1:19,19:46] = im'
        imbig = imresize(impad,(128,128))
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_ang_1_1_wd1.h5", "main/data", mnist_DHC_out)

Progress: 100%|█████████████████████████████████████████| Time: 0:38:32
